### Experiment 1

In [50]:
import numpy as np
import pathlib
import json
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

#### Directories

In [18]:
work_dir = pathlib.Path("data/example")

file_id = "0000917b-dd07-4c87-8486-cdc017a98679"
f_json = work_dir.joinpath(f"{file_id}.json")
f_jpg =  work_dir.joinpath(f"{file_id}.jpg")

In [ ]:
out_dir = pathlib.Path("data/experiment_1")
out_dir.mkdir(parents=True, exist_ok=True)

#### Annotation

In [54]:
d_json = json.load(f_json.open("r"))
d_json

[{'licensePlateLocation': [{'x': 432.78521728515625, 'y': 1374.353759765625},
   {'x': 400.15679931640625, 'y': 1470.9892578125},
   {'x': 511.45330810546875, 'y': 1496.2449951171875},
   {'x': 539.7349243164062, 'y': 1394.3443603515625}],
  'alprMeta': {'licensePlateState': 'Washington',
   'licensePlateText': 'BBX8601',
   'vehicleGroup': 'RegularCar',
   'metadataTag': []}},
 {'licensePlateLocation': [{'x': 2269.83642578125, 'y': 1213.057373046875},
   {'x': 2265.05615234375, 'y': 1235.85498046875},
   {'x': 2302.56201171875, 'y': 1239.5321044921875},
   {'x': 2306.974609375, 'y': 1215.9990234375}],
  'alprMeta': {'licensePlateState': None,
   'licensePlateText': None,
   'vehicleGroup': 'RegularCar',
   'metadataTag': []}}]

#### Mark license plates polylines

In [42]:
frame = cv2.imread(str(f_jpg))
w,h = frame.shape[1], frame.shape[0]
print(frame.shape)

color = (0,255,0) # BGR
isClosed = True
thickness = 2

for ann in d_json:
    pts = np.array([[p['x'], p['y']] for p in ann['licensePlateLocation']], np.int32)
    pts = pts.reshape((-1, 1, 2))
    image = cv2.polylines(frame, [pts], isClosed, color, thickness)
    #image = cv2.rectangle(frame, (t['x1'], t['y1']), (t['x1'] + t['w'], t['y1'] + t['h']), color, 1)
cv2.imwrite(str(out_dir.joinpath(f"{file_id}_marked.jpg")), image)

True

#### Warp polygon into rectangle

In [43]:
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")
    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    # return the ordered coordinates
    return rect

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")
    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    # return the warped image
    return warped

In [46]:
ann = d_json[0]
pts = np.array([[p['x'], p['y']] for p in ann['licensePlateLocation']], np.int32)
warped = four_point_transform(image, pts)
cv2.imwrite(str(out_dir.joinpath(f"{file_id}_0.jpg")), warped)

True

#### Transform color image to gray

In [49]:
frame = cv2.imread(str(f_jpg))
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cv2.imwrite(str(out_dir.joinpath(f"{file_id}_gray.jpg")), gray)

True

#### Canny Edge Detection
https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_canny/py_canny.html

In [55]:
img = cv2.imread(str(f_jpg))
w,h = img.shape[1], img.shape[0]
edges = cv2.Canny(img,100,200)
cv2.imwrite(str(out_dir.joinpath(f"{file_id}_edges.jpg")), edges)

True

#### Get all files id dir and subdir

In [77]:
import os
from pathlib import Path

my_dir = "/Users/silinskiy/mydata/ALPR/scaleai"

all_files = list()
for (dirpath, dirnames, filenames) in os.walk(my_dir):
    all_files += [str(Path(dirpath).joinpath(file)) for file in filenames if file.endswith('.jpg')]

In [78]:
listOfFiles

['/Users/silinskiy/mydata/ALPR/scaleai/797/797e97b1-01ac-45e0-af44-538c0056864d.jpg',
 '/Users/silinskiy/mydata/ALPR/scaleai/797/797d1168-41e6-49f3-b96f-c6ed6dccbef3.jpg',
 '/Users/silinskiy/mydata/ALPR/scaleai/797/797c09a9-14bb-44dd-aed8-82b6d63d63ca.jpg',
 '/Users/silinskiy/mydata/ALPR/scaleai/797/79776bd9-4339-4d73-be0a-e976a2c54fd2.jpg',
 '/Users/silinskiy/mydata/ALPR/scaleai/797/797c4326-e7d0-4224-b48b-4769860f0204.jpg',
 '/Users/silinskiy/mydata/ALPR/scaleai/797/797b93ef-8d59-4599-b79a-d05996e22d40.jpg',
 '/Users/silinskiy/mydata/ALPR/scaleai/797/7974baf1-979c-46be-894b-fb4ecfabc3e6.jpg',
 '/Users/silinskiy/mydata/ALPR/scaleai/eac/eac67f1a-5915-4b20-b904-4a5232a12740.jpg',
 '/Users/silinskiy/mydata/ALPR/scaleai/eac/eac3234e-0f9f-4ebe-8691-d88b33ee29ce.jpg',
 '/Users/silinskiy/mydata/ALPR/scaleai/eac/eac0db84-4f04-4176-a9c4-401e6e1d10b9.jpg',
 '/Users/silinskiy/mydata/ALPR/scaleai/eac/eac14713-eb3d-4288-9b3c-295b098b1182.jpg',
 '/Users/silinskiy/mydata/ALPR/scaleai/39f/39f26ce8-e6